# Installs

In [ ]:
!pip install --quiet geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.2 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/weather_data/2018.zip --quiet
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/weather_data/2019.zip --quiet
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/weather_data/2020.zip --quiet
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/weather_data/2021.zip --quiet
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/weather_data/2022.zip --quiet
!wget https://github.com/9r4vEy4r6/host/raw/main/telangana_ac.geojson --quiet
!unzip -q 2018.zip -d 2018/
!unzip -q 2019.zip -d 2019/
!unzip -q 2020.zip -d 2020/
!unzip -q 2021.zip -d 2021/
!unzip -q 2022.zip -d 2022/

# Imports

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Path, PathPatch
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from shapely.geometry import Point
import os
import json
import subprocess
import gc

# Load Data

In [ ]:
data = gpd.read_file("telangana_ac.geojson")
data.to_crs(crs=3857) 
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
YEARS = ["2018","2019","2020","2021","2022"]
# YEARS = ["2019","2020","2021","2022"]
FEATURES = ['Rain', 'Min_Temp', 'Max_Temp', 
            'Min_Humidity', 'Max_Humidity', 
            'Min_Wind_Speed', 'Max_Wind_Speed']
# FEATURES = ['Min_Temp', 'Max_Temp']

GLOBAL_SCALE = {}
for feature in FEATURES :
  GLOBAL_SCALE[feature] = {
      "min" : float("inf"),
      "max" : 0
  }

for folder in YEARS :
  csvs = os.listdir(folder)
  for csv in csvs :
    weather = pd.read_csv(f"{folder}/{csv}")
    weather = weather[["Date", *FEATURES, "latitude", "longitude"]]
    
    for feature in FEATURES : 
      try :
        min = float(weather[feature].min())
        max = float(weather[feature].max())
        if min < GLOBAL_SCALE[feature]["min"] : 
          GLOBAL_SCALE[feature]["min"] = min
        if max > GLOBAL_SCALE[feature]["max"] : 
          GLOBAL_SCALE[feature]["max"] = max
      except :
        pass

In [ ]:
GLOBAL_SCALE

{'Rain': {'min': 0.0, 'max': 391.0},
 'Min_Temp': {'min': 2.7, 'max': 87.6},
 'Max_Temp': {'min': -1.0, 'max': 47.9},
 'Min_Humidity': {'min': -1.0, 'max': 100.0},
 'Max_Humidity': {'min': -1.0, 'max': 100.0},
 'Min_Wind_Speed': {'min': -1.0, 'max': 31.3},
 'Max_Wind_Speed': {'min': -1.0, 'max': 234.4}}

In [ ]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


In [ ]:
with open("images/scales.json", "w") as f :
  json.dump(GLOBAL_SCALE, f, indent=4)

In [ ]:
for feature in FEATURES : 
  subprocess.run(["mkdir", f"images/{feature}"])

In [ ]:
def func(data_map, row) :
  observations = []
  for i in data_map : 
    if row["geometry"].contains(i) :
      observations.append(data_map[i])
  if len(observations) == 0 : 
    return float('nan')
  else :
    return sum(observations)/len(observations)

for folder in YEARS :
  csvs = os.listdir(folder)
  for csv in csvs : 
    for feature in FEATURES : 
      print(folder, csv, feature)
      weather = pd.read_csv(f"{folder}/{csv}")
      for date in tqdm(weather["Date"].unique()) :  
        day_datas = weather[weather["Date"]==date].drop(["Date"], axis=1)[[feature, "latitude", "longitude"]].copy()
        data_map = {}
        for day_data in day_datas.to_dict("records") : 
          if day_data[feature] != day_data[feature] or day_data["latitude"] != day_data["latitude"] or day_data["longitude"] != day_data["longitude"] :
              continue
          data_map[Point(day_data["longitude"], day_data["latitude"])] = day_data[feature]
        d = data.copy()
        fig, ax = plt.subplots(figsize=(10,10))
        d.boundary.plot(ax=ax, color='black', alpha = 0.8, linewidth=1, edgecolor='black')
        d["new_gen"] = d.apply(lambda x : func(data_map, x), axis=1)
        d[d["new_gen"].isna() == False]
        plot = d.plot(column="new_gen", vmin=GLOBAL_SCALE[feature]["min"], vmax=GLOBAL_SCALE[feature]["max"], ax=ax)
        plt.tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)
        plot.figure.savefig(f"images/{feature}/{date}.png")
        plt.close()
        del d
        del day_datas
        del data_map
        gc.collect()
      del weather
      gc.collect()
    break

2018 2018.csv Rain


100%|██████████| 365/365 [20:59<00:00,  3.45s/it]


2018 2018.csv Min_Temp


 37%|███▋      | 135/365 [08:54<16:17,  4.25s/it]

In [ ]:
!zip -r weather-images.zip images/* --quiet

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!cp weather-images.zip /gdrive/MyDrive/taim_datasets/